# Eat Safe, Love

## Notebook Set Up

In [4]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [5]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [6]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [8]:
# review the collections in our database
collections = db.list_collection_names()
print("Collections in the uk_food database:")
for collection in collections:
    print(collection)


Collections in the uk_food database:
establishments


In [12]:
# assign the collection to a variable
establishments_collection = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [13]:
# Find the establishments with a hygiene score of 20
query = query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
hygiene_20_count = establishments_collection.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {hygiene_20_count}")

# Display the first document in the results using pprint
hygiene_20_document = establishments_collection.find_one(query)
pprint(hygiene_20_document)

Number of establishments with a hygiene score of 20: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('666dc783d1762bc709f48f23'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'

In [15]:
# Convert the result to a Pandas DataFrame
hygiene_20_df = pd.DataFrame(list(hygiene_20_document))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(hygiene_20_df)}")

# Display the first 10 rows of the DataFrame
hygiene_20_df.head(10)


Number of rows in the DataFrame: 28


,0
0,_id
1,FHRSID
2,ChangesByServerID
3,LocalAuthorityBusinessID
4,BusinessName
5,BusinessType
6,BusinessTypeID
7,AddressLine1
8,AddressLine2
9,AddressLine3


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [16]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.

query = {
    "LocalAuthorityName": {"$regex": "London", "$options": "i"},
    "RatingValue": {"$gte": 4}
}

# Use count_documents to display the number of documents in the result
london_rating_count = establishments_collection.count_documents(query)
print(f"Number of establishments in London with a RatingValue >= 4: {london_rating_count}")

# Display the first document in the results using pprint
london_rating_document = establishments_collection.find_one(query)
pprint(london_rating_document)


Number of establishments in London with a RatingValue >= 4: 33
{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('666dc783d1762bc709f4a93f'),
 'geocode': {'latitude': 51.369321, 'longitude': 0.508551},
 'links': [{'href': 'https://api.ratings.

In [19]:
# Convert the result to a Pandas DataFrame
london_rating_df = pd.DataFrame(list(london_rating_document))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(london_rating_df)}")

# Display the first 10 rows of the DataFrame
london_rating_df.head(10)


Number of rows in the DataFrame: 28


,0
0,_id
1,FHRSID
2,ChangesByServerID
3,LocalAuthorityBusinessID
4,BusinessName
5,BusinessType
6,BusinessTypeID
7,AddressLine1
8,AddressLine2
9,AddressLine3


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [21]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.083840

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}

sort = [("scores.Hygiene", 1)]

limit = 5

results = establishments_collection.find(query).sort(sort).limit(limit)
results_df = pd.DataFrame(list(results))

# Print the results
print(f"Number of rows in the DataFrame: {len(results_df)}")
results_df.head(10)

Number of rows in the DataFrame: 5


,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links
0,666dc783d1762bc709f4e760,1380578,0,14425,Howe and Co Fish and Chips - Van 17,Mobile caterer,7846,Restaurant And Premises 107A Plumstead High St...,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.0925370007753372, 'latitude': ...",,4646.955931,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
1,666dc783d1762bc709f4e79e,694478,0,PI/000086506,Atlantic Fish Bar,Takeaway/sandwich shop,7844,35 Lakedale Road,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.0912164, 'latitude': 51.4867296}",,4646.974612,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
2,666dc783d1762bc709f4e79b,695241,0,PI/000179088,Plumstead Manor Nursery,Caring Premises,5,Plumstead Manor School Old Mill Road,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.0859939977526665, 'latitude': ...",,4646.974010,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
3,666dc783d1762bc709f4e74e,695223,0,PI/000178842,Iceland,Retailers - supermarkets/hypermarkets,7840,144 - 146 Plumstead High Street,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 0.0924199968576431, 'latitude': ...",,4646.946071,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
4,666dc783d1762bc709f4e782,694609,0,PI/000116619,Volunteer,Pub/bar/nightclub,7843,130 - 132 Plumstead High Street,,Plumstead,...,http://www.royalgreenwich.gov.uk,health@royalgreenwich.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 0.09208, 'latitude': 51.4873437}",,4646.965635,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."


In [22]:
# Convert result to Pandas DataFrame
results_df = pd.DataFrame(list(results))

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [23]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]
results = list(establishments_collection.aggregate(pipeline))
results_df = pd.DataFrame(results)

# Print the number of documents in the result
print(f"Number of rows in the DataFrame: {len(results_df)}")

# Print the first 10 results
results_df.head(10)


Number of rows in the DataFrame: 55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542


In [24]:
# Convert the result to a Pandas DataFrame
results_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(results_df)}")

# Display the first 10 rows of the DataFrame
results_df.head(10)

Number of rows in the DataFrame: 55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
